In [ ]:
from PIL import Image
import numpy as np
from tqdm import tqdm
from os import listdir

In [ ]:
def overlap_score(overlapping_pixels_top, overlapping_pixels_bottom):
    score = 0
    for px1, px2 in zip(overlapping_pixels_top, overlapping_pixels_bottom):
        score += np.sum(np.abs(px1-px2))
        
    return score / (overlapping_pixels_top.shape[0] * overlapping_pixels_top.shape[1])

def find_coords(top, bottom):
    max_score = 1e9
    for x_offset in tqdm(range(-100, 100)):
        for y_offset in range(-100, 0):
            overlapping_pixels_top    = top[y_offset:, max(x_offset, 0):min(top.shape[0], x_offset+bottom.shape[0]), :]
            overlapping_pixels_bottom = bottom[:-y_offset, max(-x_offset, 0):min(bottom.shape[0], top.shape[0]-x_offset), :]
            assert overlapping_pixels_top.shape == overlapping_pixels_bottom.shape

            score = overlap_score(overlapping_pixels_top, overlapping_pixels_bottom)
            if score < max_score: 
                max_score = score
                best_coords = [x_offset, y_offset]

            if max_score == 0: break
        if max_score == 0: break

    return best_coords, max_score

In [ ]:
path = 'doku'
file_list = np.unique([s[:-6] for s in listdir(path)])
file_list

In [ ]:
%matplotlib inline

for filename in file_list:
    top = np.asarray(Image.open(path+'/'+filename+"_O.PNG"))
    bottom = np.asarray(Image.open(path+'/'+filename+"_U.PNG"))

    if top.shape[2] != 3: top = top[:, :, :3]
    if bottom.shape[2] != 3: bottom = bottom[:, :, :3]
        
    best_coords, max_score = find_coords(top, bottom)

    if max_score != 0: 
        # repeat fit with 1px removed all round both images
        top = top[1:-1, 1:-1, :]
        bottom = bottom[1:-1, 1:-1, :]
        
        best_coords, max_score = find_coords(top, bottom)
        
    if max_score != 0: print(filename)
    
    x_offset, y_offset = best_coords

    overlapping_pixels_top    = top[y_offset:, max(x_offset, 0):min(top.shape[0], x_offset+bottom.shape[0]), :]
    overlapping_pixels_bottom = bottom[:-y_offset, max(-x_offset, 0):min(bottom.shape[0], top.shape[0]-x_offset), :]
    
    #assembled_img = np.zeros(np.array(top.shape) + np.array([bottom.shape[0]+best_coords[1], max(bottom.shape[1]-top.shape[1], 0) + max(0, bottom.shape[1]-(best_coords[0]+top.shape[1])), 0]), dtype='uint8')
    assembled_img = np.zeros(np.array(top.shape) + np.array([bottom.shape[0]+best_coords[1], max(bottom.shape[1]-top.shape[1], 0) + max(0, bottom.shape[1]-top.shape[1]+best_coords[0]), 0]), dtype='uint8')
    if best_coords[0] > 0: # top image left, bottom image right
        assembled_img[0:top.shape[0], 0:top.shape[1], 0:top.shape[2]] = top

        assembled_img[top.shape[0]+best_coords[1]:top.shape[0]+best_coords[1]+bottom.shape[0], best_coords[0]:best_coords[0]+bottom.shape[1], 0:top.shape[2]] = bottom

    else: # top image right, bottom image left
        newshape = np.array(top.shape) + np.array([max(0, best_coords[0]), 0, 0])
        assembled_img[0:newshape[0], 0:newshape[1], 0:newshape[2]] = top

        newpos = np.array(top.shape) + np.array([best_coords[0], 0, 0])
        assembled_img[top.shape[0]+best_coords[1]:top.shape[0]+best_coords[1]+bottom.shape[0], :bottom.shape[1], 0:top.shape[2]] = bottom

    assembled_img_pil = Image.fromarray(assembled_img)
    assembled_img_pil.save(path+'/stitched/'+filename+".PNG")
    #display(assembled_img_pil)